In [1]:
import os
import time

import numpy as np
import pandas as pd
import tensorflow as tf
from keras.callbacks import History
from keras.callbacks import TensorBoard
from keras.layers import Dense
from keras.models import Sequential
from tensorflow import keras

2024-06-07 03:02:13.309340: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-07 03:02:13.591920: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-06-07 03:02:13.591946: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2024-06-07 03:02:14.769572: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2024-

In [2]:
np.random.seed(42)
tf.random.set_seed(42)

Wczytanie danych, usunięcie brakujących wartości, nadanie etykiet

In [3]:
train_set = pd.read_csv('adult/adult.data', sep=", ", header=None, engine='python')
test_set = pd.read_csv('adult/adult.test', sep=", ", skiprows=1, header=None, engine='python')

col_labels = ['age', 'workclass', 'fnlwgt', 'education', 'education_num', 'marital_status', 'occupation',
              'relationship', 'race', 'sex', 'capital_gain', 'capital_loss', 'hours_per_week', 'native_country',
              'wage_class']

train_set.columns = col_labels
test_set.columns = col_labels

train = train_set.replace('?', np.nan).dropna()
test = test_set.replace('?', np.nan).dropna()

train_set.head()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,wage_class
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


# Zad.
Zróbmy szybki preprocesing

In [4]:
dataset = pd.concat([train, test])

dataset['wage_class'] = dataset.wage_class.replace({'<=50K.': 0, '<=50K': 0, '>50K.': 1, '>50K': 1})

dataset.drop(["fnlwgt"], axis=1, inplace=True)
dataset.drop(["education"], axis=1, inplace=True)

x = dataset.groupby('native_country')["wage_class"].mean()
d = dict(pd.cut(x[x.index != " United-States"], 5, labels=range(5)))
dataset['native_country'] = dataset['native_country'].replace(d)

dataset = pd.get_dummies(dataset, drop_first=True)

train = dataset.iloc[:train.shape[0]]
test = dataset.iloc[train.shape[0]:]

X_train = train.drop("wage_class", axis=1)
y_train = train.wage_class

X_test = test.drop("wage_class", axis=1)
y_test = test.wage_class

# TensorBoard

In [5]:
root_logdir = os.path.join(os.curdir, "my_logs")

In [6]:
def get_run_logdir():
    run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
    return os.path.join(root_logdir, run_id)


run_logdir = get_run_logdir()
run_logdir

'./my_logs/run_2024_06_07-03_02_18'

In [7]:
history = History()
model = Sequential()
model.add(Dense(100, activation="sigmoid", input_shape=(X_train.shape[1],)))
model.add(Dense(50, activation="sigmoid"))
model.add(Dense(10, activation="sigmoid"))
model.add(Dense(1, activation="sigmoid"))
model.summary()

2024-06-07 03:02:18.140953: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2024-06-07 03:02:18.141004: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2024-06-07 03:02:18.141038: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (kjUbuntu): /proc/driver/nvidia/version does not exist
2024-06-07 03:02:18.141450: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100)               4200      
                                                                 
 dense_1 (Dense)             (None, 50)                5050      
                                                                 
 dense_2 (Dense)             (None, 10)                510       
                                                                 
 dense_3 (Dense)             (None, 1)                 11        
                                                                 
Total params: 9,771
Trainable params: 9,771
Non-trainable params: 0
_________________________________________________________________


In [8]:
model.compile(loss="binary_crossentropy", optimizer="Adam", metrics=["accuracy"])

In [9]:
X_train = np.asarray(X_train).astype('float32')
y_train = np.asarray(y_train).astype('float32')
X_test = np.asarray(X_test).astype('float32')
y_test = np.asarray(y_test).astype('float32')

tensorboard_cb = TensorBoard(run_logdir)
history = model.fit(X_train, y_train, epochs=30, validation_data=(X_test, y_test), callbacks=[tensorboard_cb])

Epoch 1/30
943/943 [==============================] - 8s 7ms/step - loss: 0.4326 - accuracy: 0.7914 - val_loss: 0.3917 - val_accuracy: 0.8068
Epoch 2/30
943/943 [==============================] - 7s 7ms/step - loss: 0.3769 - accuracy: 0.8130 - val_loss: 0.3716 - val_accuracy: 0.8139
Epoch 3/30
943/943 [==============================] - 6s 7ms/step - loss: 0.3737 - accuracy: 0.8133 - val_loss: 0.3699 - val_accuracy: 0.8177
Epoch 4/30
943/943 [==============================] - 6s 6ms/step - loss: 0.3713 - accuracy: 0.8158 - val_loss: 0.4049 - val_accuracy: 0.8039
Epoch 5/30
943/943 [==============================] - 6s 7ms/step - loss: 0.3697 - accuracy: 0.8144 - val_loss: 0.3685 - val_accuracy: 0.7936
Epoch 6/30
943/943 [==============================] - 6s 7ms/step - loss: 0.3686 - accuracy: 0.8133 - val_loss: 0.3721 - val_accuracy: 0.8070
Epoch 7/30
943/943 [==============================] - 6s 6ms/step - loss: 0.3679 - accuracy: 0.8177 - val_loss: 0.3631 - val_accuracy: 0.8210
Epoch 

o start the TensorBoard server, one option is to open a terminal, if needed activate the virtualenv where you installed TensorBoard, go to this notebook's directory, then type:

```python
$ tensorboard --logdir=./my_logs --port=6006
```

You can then open your web browser to localhost:6006 and use TensorBoard. Once you are done, press Ctrl-C in the terminal window, this will shutdown the TensorBoard server.

Alternatively, you can load TensorBoard's Jupyter extension and run it like this:

```python
%load_ext tensorboard
%tensorboard --logdir=./my_logs --port=6006
```

Gdyby nie działał tensorboard

https://github.com/pytorch/pytorch/issues/22676
```python
pip uninstall tb-nightly tensorboardX tensorboard
pip install tensorboard
```

# Zad

* Naucz dowolny model na zbiorze MNIST.
* Wyświetl krzywe uczenia w tensorboard

In [10]:
mnist = keras.datasets.mnist
(X_train_full, y_train_full), (X_test, y_test) = mnist.load_data()
X_train_full = X_train_full.astype(np.float32) / 255
X_test = X_test.astype(np.float32) / 255
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

In [11]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, activation="relu"),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.Dense(10, activation="softmax")
])
model.compile(loss="sparse_categorical_crossentropy",
              optimizer="sgd",
              metrics=["accuracy"])

In [12]:
root_logdir = os.path.join(os.curdir, "my_logs_mnist")

In [13]:
tensorboard_cb = TensorBoard(run_logdir)

history = model.fit(X_train, y_train, epochs=30,
                    validation_data=(X_valid, y_valid),
                    callbacks=[tensorboard_cb])


Epoch 1/30


2024-06-07 03:05:12.505401: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 172480000 exceeds 10% of free system memory.


1719/1719 [==============================] - 13s 7ms/step - loss: 0.5912 - accuracy: 0.8489 - val_loss: 0.3064 - val_accuracy: 0.9166
Epoch 2/30
1719/1719 [==============================] - 13s 8ms/step - loss: 0.2875 - accuracy: 0.9184 - val_loss: 0.2460 - val_accuracy: 0.9304
Epoch 3/30
1719/1719 [==============================] - 12s 7ms/step - loss: 0.2373 - accuracy: 0.9324 - val_loss: 0.2049 - val_accuracy: 0.9424
Epoch 4/30
1719/1719 [==============================] - 14s 8ms/step - loss: 0.2028 - accuracy: 0.9429 - val_loss: 0.1808 - val_accuracy: 0.9480
Epoch 5/30
1719/1719 [==============================] - 12s 7ms/step - loss: 0.1770 - accuracy: 0.9503 - val_loss: 0.1621 - val_accuracy: 0.9546
Epoch 6/30
1719/1719 [==============================] - 12s 7ms/step - loss: 0.1570 - accuracy: 0.9564 - val_loss: 0.1472 - val_accuracy: 0.9604
Epoch 7/30
1719/1719 [==============================] - 12s 7ms/step - loss: 0.1407 - accuracy: 0.9604 - val_loss: 0.1369 - val_accuracy: 0.9

```
$ tensorboard --logdir=./my_logs --port=6006
```